# Creating a single event-log

In [18]:
# Install requirements
#!pip install sybps
#!pip install numpy pandas pm4py
#!brew install graphviz

In [19]:
import sys
import os
import numpy as np
sys.path.append(os.path.abspath('../src'))

In [20]:
eventlog_settings = {
                    # number of traces/cases in the event-log
                    "number_of_traces":1000,

                    # level of entropy: min, medium and/or max
                    "process_entropy": "max_entropy",#"min_entropy","med_entropy","max_entropy"

                    # first or higher-order markov chain to represent the transitions "memoryless", "memory"
                    "process_type":"memory",#"memoryless",

                    # order of the HOMC - only specify this when using process with memory
                    "process_memory":2,

                    # number of activity types
                    "statespace_size":5,
                    
                    # number of transitions - only used for medium entropy (should be higher than 2 and < statespace size)
                    "med_ent_n_transitions":3,
                                    
                    # lambda parameter of inter-arrival times
                    "inter_arrival_time":1.5,
                    
                    # lambda parameter of process noise
                    "process_stability_scale":0.1,
                    
                    # probability of agent being available
                    "resource_availability_p":0.5,

                    # number of agents in the process
                    "resource_availability_n":3,

                    # waiting time in full days, when no agent is available. 0.041 is 15 minutes
                    "resource_availability_m":0.041,
                    
                    # variation between activity durations
                    "activity_duration_lambda_range":1,
                    
                    # business hours definition: when can cases be processed? ()
                    "Deterministic_offset_W":"weekdays",

                    # time-unit for a full week: days = 7, hrs = 24*7, etc.
                    "Deterministic_offset_u":7,

                    # offset for the timestamps used (years after 1970)
                    "datetime_offset":54,

                    # seed value for replication: Set this to a fixed value if the results should be reproducible
                    #"seed_value":int(np.random.uniform(low=0, high=2**32 - 1))
                    "seed_value":1337
                    }

In [21]:
from SynBPS.simulation.eventlog import generate_eventlog

In [22]:
log = generate_eventlog(eventlog_settings)

seed: 1337
generated traces: 1000
traces: 1000
events: 5467
ids: 1000


In [23]:
print(len(log)) 
print(np.sum(log.z_t)) 
print(np.sum(log.u_t)) 
print(log.activity.value_counts()[0]) 

5467
4075334.6217161743
2536.7777076000366
1245


/var/folders/f6/x71hwvz92j5gxx_skcp07qth0000gn/T/ipykernel_33457/1851877283.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(log.activity.value_counts()[0])


In [24]:
seed = 1
4561
3435319.5044537717
2034.6863010385719
1226

1226

In [25]:
log

,caseid,activity,activity_no,y_acc_sum,z_t,n_t,q_t,h_t,b_t,s_t,v_t,u_t,starttime,endtime,arrival_datetime,start_datetime,end_datetime,start_day,start_hour
0,0,d,1,0,0.455767,0.455767,0.514046,0.041,0.017279,0.000000,0.146338,0.204617,0.514046,0.660384,2024-01-02 04:56:18,2024-01-02 06:20:13,2024-01-02 09:50:57,Tuesday,6
1,0,f,2,0,0.455767,0.660384,0.718663,0.041,0.017279,0.000000,0.311423,0.369702,0.718663,1.030086,2024-01-02 04:56:18,2024-01-02 11:14:52,2024-01-02 18:43:19,Tuesday,11
2,0,c,3,0,0.455767,1.030086,1.088365,0.041,0.017279,0.411635,0.122181,0.592095,1.500000,1.622181,2024-01-02 04:56:18,2024-01-03 06:00:00,2024-01-03 08:55:56,Wednesday,6
3,1,b,1,0,0.714949,0.714949,0.773228,0.041,0.017279,0.000000,0.085419,0.143698,0.773228,0.858647,2024-01-02 11:09:31,2024-01-02 12:33:26,2024-01-02 14:36:27,Tuesday,12
4,1,c,2,0,0.714949,0.858647,0.916926,0.041,0.017279,0.000000,0.041299,0.099578,0.916926,0.958226,2024-01-02 11:09:31,2024-01-02 16:00:22,2024-01-02 16:59:50,Tuesday,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5462,996,e,10,0,1494.581472,1499.929547,1.987826,0.041,0.017279,0.000000,0.040932,0.099211,1499.987826,1500.028758,2028-02-04 07:57:19,2028-02-09 17:42:28,2028-02-09 18:41:24,Wednesday,17
5463,997,b,1,0,1495.389588,1495.389588,4.447867,0.041,0.017279,0.052133,0.085419,0.195831,1495.500000,1495.585419,2028-02-05 03:21:00,2028-02-05 06:00:00,2028-02-05 08:03:00,Saturday,6
5464,998,f,1,0,1497.458679,1497.458679,6.516958,0.041,0.017279,0.983042,0.026311,1.067631,1498.500000,1498.526311,2028-02-07 05:00:29,2028-02-08 06:00:00,2028-02-08 06:37:53,Tuesday,6
5465,999,f,1,0,1499.887822,1499.887822,1.946101,0.041,0.017279,0.000000,0.026311,0.084590,1499.946101,1499.972412,2028-02-09 15:18:27,2028-02-09 16:42:23,2028-02-09 17:20:16,Wednesday,16


# BPMN model

# Variant analysis